In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode

In [ ]:
%%capture
%run "12. Agrupamientos de variables socioeconomicas.ipynb"

In [3]:
Variables_A_Dummear = [
    'Genero',
    'Estrato_Social',
    'Nivel_Educativo',
    'Fuente_Ingreso',
    'Inmueble_Residencia',
    'Voto_2019',
    'Voto_PASO_2023',
    'Candidato_PASO_2023',
    'Afiliacion_Politica',
    'Edad_Agrupada',
    'Votara_2023',
    'Autopercepcion_Izq_Der_Agrupada',
    'Autopercepcion_Con_Pro_Agrupada',
    'Autopercepcion_Per_Antiper_Agrupada',
    'Region',
    'Categoria_PASO_2023'
]

In [4]:
# Crear variables dummy para todas las variables categóricas.
#print("Creando variables dummy...")

for Nombre_df, df in dfs_Finales.items():
   #print(f"\nProcesando: {Nombre_df}")
   
   # Verificar qué variables existen en este DataFrame.
   Variables_Existentes = [var for var in Variables_A_Dummear if var in df.columns]
   Variables_Faltantes = [var for var in Variables_A_Dummear if var not in df.columns]
   
   #print(f"  Variables encontradas: {len(Variables_Existentes)}")
   #print(f"  Variables faltantes: {len(Variables_Faltantes)}")
   
   #if Variables_Faltantes:
       #print(f"  Faltantes: {Variables_Faltantes}")
   
   # Crear dummies para las variables existentes.
   if Variables_Existentes:
       for Variable in Variables_Existentes:
           #print(f"    Creando dummies para: {Variable}")
           
           # Contar valores únicos antes.
           Valores_Unicos = df[Variable].value_counts(dropna=False)
           #print(f"      Valores únicos: {len(Valores_Unicos)}")
           
           # Crear variables dummy.
           Dummies = pd.get_dummies(df[Variable], prefix=Variable, dummy_na=False)
           
           # Agregar las columnas dummy al DataFrame.
           df = pd.concat([df, Dummies], axis=1)
           
           #print(f"      Columnas dummy creadas: {len(Dummies.columns)}")
       
       # Actualizar el DataFrame en el diccionario.
       dfs_Finales[Nombre_df] = df
       
       #print(f"  Total de columnas después: {len(df.columns)}")

#print("\nCreación de variables dummy completada.")

In [ ]:
print("="*80)
print("VERIFICACIÓN: CREACIÓN DE VARIABLES DUMMY")
print("="*80)

# 1. Verificar existencia de la lista de variables.
print("1. VERIFICACIÓN DE LISTA DE VARIABLES A DUMMEAR:")

if 'Variables_A_Dummear' in globals():
    print(f"   ✅ Lista definida con {len(Variables_A_Dummear)} variables:")
    for i, Variable in enumerate(Variables_A_Dummear, 1):
        print(f"      {i:2}. {Variable}")
else:
    print("   ❌ Lista 'Variables_A_Dummear' no encontrada")
    # Definir manualmente.
    Variables_A_Dummear = [
        'Genero', 'Estrato_Social', 'Nivel_Educativo', 'Fuente_Ingreso',
        'Inmueble_Residencia', 'Voto_2019', 'Voto_PASO_2023', 'Candidato_PASO_2023',
        'Afiliacion_Politica', 'Edad_Agrupada', 'Votara_2023', 'Autopercepcion_Izq_Der_Agrupada',
        'Autopercepcion_Con_Pro_Agrupada', 'Autopercepcion_Per_Antiper_Agrupada', 'Region',
        'Categoria_PASO_2023'
    ]
    print(f"   🔧 Definiendo manualmente: {len(Variables_A_Dummear)} variables")

print("\n" + "="*60)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n🎯 DATAFRAME: {Nombre_df}")
    print("-" * 50)
    
    Total_Casos = len(df)
    Total_Columnas_Original = len(df.columns)
    
    # 2. Identificar variables dummy creadas.
    print("2. IDENTIFICACIÓN DE VARIABLES DUMMY:")
    
    Variables_Dummy_Encontradas = {}
    Columnas_Dummy_Total = 0
    
    for Variable in Variables_A_Dummear:
        if Variable in df.columns:
            # Buscar columnas que empiecen con el prefijo de esta variable.
            Prefijo = f"{Variable}_"
            Columnas_Dummy = [col for col in df.columns if col.startswith(Prefijo)]
            Variables_Dummy_Encontradas[Variable] = Columnas_Dummy
            Columnas_Dummy_Total += len(Columnas_Dummy)
            
            print(f"   {Variable}:")
            print(f"     Variable original: {'✅' if Variable in df.columns else '❌'}")
            print(f"     Columnas dummy: {len(Columnas_Dummy)}")
            if len(Columnas_Dummy) <= 10:  # Mostrar todas si son pocas.
                for Dummy in Columnas_Dummy:
                    print(f"       - {Dummy}")
            else:  # Mostrar solo las primeras si son muchas.
                for Dummy in Columnas_Dummy[:5]:
                    print(f"       - {Dummy}")
                print(f"       ... y {len(Columnas_Dummy) - 5} más")
        else:
            print(f"   {Variable}: ❌ Variable original no encontrada")
    
    print(f"\n   📊 RESUMEN:")
    print(f"     Variables procesadas: {len(Variables_Dummy_Encontradas)}/{len(Variables_A_Dummear)}")
    print(f"     Columnas dummy creadas: {Columnas_Dummy_Total}")
    print(f"     Columnas totales: {Total_Columnas_Original}")
    
    # 3. Verificar integridad de variables dummy.
    print("\n3. VERIFICACIÓN DE INTEGRIDAD:")
    
    for Variable, Columnas_Dummy in Variables_Dummy_Encontradas.items():
        print(f"\n   🔍 {Variable}:")
        
        if len(Columnas_Dummy) == 0:
            print("     ❌ No se encontraron columnas dummy")
            continue
        
        # Verificar que todas las columnas dummy sean binarias.
        Columnas_No_Binarias = []
        for Dummy in Columnas_Dummy:
            Valores_Unicos = df[Dummy].unique()
            Valores_Validos = [v for v in Valores_Unicos if pd.notna(v)]
            if not set(Valores_Validos).issubset({0, 1}):
                Columnas_No_Binarias.append(Dummy)
        
        if Columnas_No_Binarias:
            print(f"     ❌ Columnas no binarias: {Columnas_No_Binarias}")
        else:
            print("     ✅ Todas las columnas son binarias [0,1]")
        
        # Verificar que la suma por fila sea consistente.
        if len(Columnas_Dummy) > 1:
            Suma_Por_Fila = df[Columnas_Dummy].sum(axis=1)
            Valores_Suma_Unicos = Suma_Por_Fila.unique()
            
            print(f"     Suma por fila: {sorted(Valores_Suma_Unicos)}")
            
            # Para variables categóricas normales, cada fila debería sumar 1.
            if set(Valores_Suma_Unicos) == {1}:
                print("     ✅ Cada fila suma exactamente 1")
            elif 0 in Valores_Suma_Unicos and 1 in Valores_Suma_Unicos:
                Filas_Cero = (Suma_Por_Fila == 0).sum()
                print(f"     ⚠️  {Filas_Cero} filas suman 0 (valores NaN en original)")
            else:
                print(f"     ❌ Sumas inconsistentes: {Valores_Suma_Unicos}")
        
        # Verificar coherencia con variable original.
        if Variable in df.columns:
            Categorias_Originales = set(df[Variable].dropna().unique())
            Categorias_Dummy = set()
            
            for Dummy in Columnas_Dummy:
                # Extraer la categoría del nombre de la columna dummy.
                Categoria = Dummy.replace(f"{Variable}_", "")
                Categorias_Dummy.add(Categoria)
            
            Categorias_Faltantes = Categorias_Originales - Categorias_Dummy
            Categorias_Extras = Categorias_Dummy - Categorias_Originales
            
            if not Categorias_Faltantes and not Categorias_Extras:
                print("     ✅ Perfecta correspondencia con variable original")
            else:
                if Categorias_Faltantes:
                    print(f"     ⚠️  Categorías faltantes: {Categorias_Faltantes}")
                if Categorias_Extras:
                    print(f"     ⚠️  Categorías extras: {Categorias_Extras}")
    
    # 4. VERIFICACIÓN MANUAL CON CASOS ESPECÍFICOS.
    print("\n4. VERIFICACIÓN MANUAL - CASOS ESPECÍFICOS:")
    
    # Seleccionar 4 casos al azar para verificación manual.
    if len(df) >= 4:
        import random
        random.seed(789)  # Para reproducibilidad.
        Indices_Muestra = random.sample(range(len(df)), min(4, len(df)))
        
        # Seleccionar 3 variables principales para verificar.
        Variables_A_Verificar = ['Genero', 'Region', 'Nivel_Educativo']
        Variables_Disponibles = [v for v in Variables_A_Verificar if v in Variables_Dummy_Encontradas]
        
        for i, Indice in enumerate(Indices_Muestra, 1):
            Fila = df.iloc[Indice]
            ID_Participante = Fila['ID']
            
            print(f"\n   🔍 CASO {i} - ID: {ID_Participante}")
            
            for Variable in Variables_Disponibles[:2]:  # Solo 2 para no saturar.
                if Variable in df.columns:
                    Valor_Original = Fila[Variable]
                    print(f"      {Variable}: '{Valor_Original}'")
                    
                    # Verificar columnas dummy correspondientes.
                    Columnas_Dummy = Variables_Dummy_Encontradas.get(Variable, [])
                    Dummies_Activos = []
                    
                    for Dummy in Columnas_Dummy:
                        if Fila[Dummy] == 1:
                            Categoria = Dummy.replace(f"{Variable}_", "")
                            Dummies_Activos.append(Categoria)
                    
                    print(f"        Dummies activos: {Dummies_Activos}")
                    
                    # Verificar coherencia.
                    if len(Dummies_Activos) == 1 and str(Valor_Original) == str(Dummies_Activos[0]):
                        print("        ✅ Coherencia perfecta")
                    elif len(Dummies_Activos) == 0 and pd.isna(Valor_Original):
                        print("        ✅ Coherencia (valor NaN)")
                    else:
                        print(f"        ❌ Incoherencia detectada")
    
    # 5. Análisis de distribución de categorías.
    print("\n5. ANÁLISIS DE DISTRIBUCIÓN:")
    
    Variables_Principales = ['Genero', 'Region', 'Nivel_Educativo', 'Estrato_Social']
    Variables_Principales_Disponibles = [v for v in Variables_Principales if v in Variables_Dummy_Encontradas]
    
    for Variable in Variables_Principales_Disponibles:
        print(f"\n   📊 {Variable}:")
        
        Columnas_Dummy = Variables_Dummy_Encontradas[Variable]
        if len(Columnas_Dummy) > 0:
            # Calcular frecuencias de cada categoría.
            Distribuciones = []
            for Dummy in Columnas_Dummy:
                Categoria = Dummy.replace(f"{Variable}_", "")
                Frecuencia = df[Dummy].sum()
                Porcentaje = (Frecuencia / Total_Casos) * 100
                Distribuciones.append((Categoria, Frecuencia, Porcentaje))
            
            # Ordenar por frecuencia.
            Distribuciones.sort(key=lambda x: x[1], reverse=True)
            
            print("     Categoría               | Casos | % Total")
            print("     " + "-"*45)
            
            for Categoria, Frecuencia, Porcentaje in Distribuciones:
                print(f"     {Categoria:<23} | {Frecuencia:5} | {Porcentaje:6.1f}%")
    
    # 6. Análisis de multicolinealidad potencial.
    print("\n6. ANÁLISIS DE MULTICOLINEALIDAD:")
    
    # Para variables categóricas normales, siempre hay multicolinealidad perfecta.
    # Una categoría es redundante (puede predecirse de las otras).
    
    print("   Variables con posible multicolinealidad perfecta:")
    for Variable, Columnas_Dummy in Variables_Dummy_Encontradas.items():
        if len(Columnas_Dummy) > 1:
            print(f"     {Variable}: {len(Columnas_Dummy)} categorías (eliminar 1 para evitar multicolinealidad)")
    
    # Verificar variables que deberían ser mutuamente excluyentes.
    Variables_Mutuamente_Excluyentes = ['Genero', 'Region', 'Nivel_Educativo',
                                         'Estrato_Social', 'Fuente_Ingreso', 
                                         'Inmueble_Residencia', 'Voto_2019', 
                                         'Voto_PASO_2023', 'Candidato_PASO_2023',
                                         'Afiliacion_Politica', 'Edad_Agrupada',
                                         'Votara_2023', 'Autopercepcion_Izq_Der_Agrupada',
                                         'Autopercepcion_Con_Pro_Agrupada',
                                         'Autopercepcion_Per_Antiper_Agrupada',
                                         'Categoria_PASO_2023']
    
    for Variable in Variables_Mutuamente_Excluyentes:
        if Variable in Variables_Dummy_Encontradas:
            Columnas_Dummy = Variables_Dummy_Encontradas[Variable]
            if len(Columnas_Dummy) > 1:
                # Verificar que no haya filas con múltiples 1s.
                Suma_Por_Fila = df[Columnas_Dummy].sum(axis=1)
                Filas_Multiples = (Suma_Por_Fila > 1).sum()
                
                if Filas_Multiples == 0:
                    print(f"   ✅ {Variable}: Categorías mutuamente excluyentes")
                else:
                    print(f"   ⚠️  {Variable}: {Filas_Multiples} filas con múltiples categorías")
    
    # 7. Impacto en dimensionalidad.
    print("\n7. IMPACTO EN DIMENSIONALIDAD:")
    
    Columnas_Antes = Total_Columnas_Original - Columnas_Dummy_Total
    Columnas_Despues = Total_Columnas_Original
    Aumento_Porcentual = ((Columnas_Despues - Columnas_Antes) / Columnas_Antes) * 100
    
    print(f"   Columnas antes del dummyficado: ~{Columnas_Antes}")
    print(f"   Columnas después: {Columnas_Despues}")
    print(f"   Aumento: +{Columnas_Dummy_Total} columnas ({Aumento_Porcentual:.1f}%)")
    
    if Aumento_Porcentual > 200:
        print("   ⚠️  Aumento significativo - considerar reducir variables")
    elif Aumento_Porcentual > 100:
        print("   📊 Aumento moderado - monitorear rendimiento de modelos")
    else:
        print("   ✅ Aumento manejable")
    
    # 8. Recomendaciones para modelado.
    print("\n8. RECOMENDACIONES PARA MODELADO:")
    
    print("   Para evitar multicolinealidad perfecta:")
    for Variable, Columnas_Dummy in Variables_Dummy_Encontradas.items():
        if len(Columnas_Dummy) > 2:
            print(f"     {Variable}: Eliminar 1 categoría (ej: {Columnas_Dummy[-1]})")
    
    # Identificar variables con muchas categorías.
    Variables_Muchas_Categorias = [(var, len(cols)) for var, cols in Variables_Dummy_Encontradas.items() if len(cols) > 10]
    
    if Variables_Muchas_Categorias:
        print("   Variables con muchas categorías (considerar agrupación):")
        for Variable, Num_Categorias in Variables_Muchas_Categorias:
            print(f"     {Variable}: {Num_Categorias} categorías")
    
    print("   Variables con distribución muy desbalanceada:")
    for Variable, Columnas_Dummy in Variables_Dummy_Encontradas.items():
        if len(Columnas_Dummy) > 1:
            # Calcular frecuencias.
            Frecuencias = [df[col].sum() for col in Columnas_Dummy]
            Frecuencia_Max = max(Frecuencias)
            Frecuencia_Min = min(Frecuencias)
            
            if Frecuencia_Min > 0:  # Evitar división por cero.
                Ratio = Frecuencia_Max / Frecuencia_Min
                if Ratio > 20:  # Más de 20:1 de desbalance.
                    print(f"     {Variable}: Ratio {Ratio:.1f}:1 (considerar agrupación)")

print("\n" + "="*80)
print("✅ VERIFICACIÓN DE VARIABLES DUMMY COMPLETADA")
print("="*80)

# 9. Resumen ejecutivo.
print("\n" + "="*80)
print("RESUMEN EJECUTIVO - VARIABLES DUMMY")
print("="*80)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n📊 DATAFRAME: {Nombre_df}")
    
    # Contar variables dummy creadas.
    Total_Variables_Procesadas = len([var for var in Variables_A_Dummear 
                                    if any(col.startswith(f"{var}_") for col in df.columns)])
    Total_Columnas_Dummy = sum(len([col for col in df.columns if col.startswith(f"{var}_")]) 
                              for var in Variables_A_Dummear)
    
    print(f"   🎯 Variables categóricas procesadas: {Total_Variables_Procesadas}")
    print(f"   📈 Columnas dummy creadas: {Total_Columnas_Dummy}")
    print(f"   📊 Total de columnas final: {len(df.columns)}")
    
    # Variable con más categorías.
    Max_Categorias = 0
    Variable_Max = ""
    for Variable in Variables_A_Dummear:
        Num_Dummies = len([col for col in df.columns if col.startswith(f"{Variable}_")])
        if Num_Dummies > Max_Categorias:
            Max_Categorias = Num_Dummies
            Variable_Max = Variable
    
    if Variable_Max:
        print(f"   🔢 Variable con más categorías: {Variable_Max} ({Max_Categorias} categorías)")
    
    print(f"   ✅ Dataset preparado para modelado con variables dummy")

print("\n" + "="*80)
print("✅ ANÁLISIS DE DUMMYFICACIÓN COMPLETADO")
print("="*80)

In [ ]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/13. Crear dummies ({Nombre_df}).xlsx', index=False)